# 1. torch.vmap
2. github链接：关于如何just use the parallelization capabilities of vmap
2. https://github.com/pytorch/functorch/issues/257

In [3]:
import torch
from functorch import vmap

In [28]:
def func_3(t,b):
    return t*b

# if分支isn't a differentiability requirement;
def func(t,b):
    tmp = t*b
    if tmp > 5:     # error: Data-dependent control flow
        root = t*b
    else:
        root = -t
    return root

def func_2(t,b):
    return torch.where((t>5.),
                        func_3(t,b),
                        -t)


t = torch.tensor([1.,2.,3.,4.,5.,6.,7.,8.])
b = torch.tensor([1.],requires_grad=True)
b_extend = torch.expand_copy(b,size=t.shape)    # 必须把b扩张到和t同一个size否则报错
b_extend.retain_grad()

print(f"shape of t:{t.shape}, shape of b_extend:{b_extend.shape}")
# shape of t:torch.Size([8]), shape of b_extend:torch.Size([8])


# Use vmap() to construct a new function.  # [D], [D] -> []
func_vec = vmap(func_2)  # [N, D], [N, D] -> [N]
ans = func_vec(t,b_extend)
ans.sum().backward()   # 等价于: ans.backward(torch.ones(b_extend.shape))
b_extend.grad          # 可以预见：b的导数是t：在t>5.时导数是t,在t<=5.时导数是0
# tensor([0., 0., 0., 0., 0., 6., 7., 8.])

shape of t:torch.Size([8]), shape of b_extend:torch.Size([8])


TypeError: func_2() missing 1 required positional argument: 'b'

# 2. np.vectorize

In [7]:
import numpy as np
def U_GT1(t, v, d, b,eps = 1e-10):
    return (v-b)*t
    # return np.clip(1 - b / (v - (t - 1) * d), eps, 1 - eps)  # 注意设置LEN之内，eps < U < 1-eps

LEN = 10
v=10
b=1
d=2
U = [0] * (LEN + 2)  # U: the prob. that someone offers a bid in t_th round
U[0], U[1] = 1., 1.  # u[0]用不到,u[1]=1保证auction至少1轮

# 使用
t = np.arange(2,LEN+1)
print(t)
U_GT1_vec = np.vectorize(U_GT1)
U[2:-1] = U_GT1_vec(t, v, d, b)
U[-1] = -1
U

[ 2  3  4  5  6  7  8  9 10]


[1.0, 1.0, 18, 27, 36, 45, 54, 63, 72, 81, 90, -1]